In [ ]:
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
from astropy.coordinates import SkyCoord
import healpy as hp
import numpy as np

In [ ]:
def slew_time(data):
    print(data['slewTime'])
    idx = data['slewTime'] < 20
    plt.hist(data[idx]['slewTime'],histtype='step',bins=40)
    print(data['slewTime'].median())
    data['time_all']=data['mjd']*24.*3600+data['visitTime']+data['slewTime']
    data['time_all']-data['mjd'][1:]*24.*3600
    data = data.sort_values(by=['mjd'])
    mjd_previous = 12
    visitTime = 34
    corr = 24.*3600.
    for i, row in data.iterrows():
        mjd= row['slewTime']/corr+mjd_previous+visitTime/corr
        print(mjd-row['mjd'])
        mjd_previous = row['mjd']
        visitTime = row['visitTime']

In [ ]:
def get_ra_dec_from_name(row):
    
    rr = None
    name = row['Simbad_Name']
    if name == 'none':
        name = row['Star_name']
    print('looking for',name)
    try:
        rr = SkyCoord.from_name(name)
    except Exception:
        name = row['Star_name']
        try:
            rr = SkyCoord.from_name(name)
        except Exception:
            rr = None
    
    return rr

In [ ]:
def get_ra_dec(ra,dec):
    rav = '{}h{}m{}s'.format(ra[0],ra[1],ra[2])
    decv = '{}d{}m{}s'.format(dec[0],dec[1],dec[2])
    c = SkyCoord(rav, decv, frame='icrs')
    return c

In [ ]:
def get_calspec(fName='../../calspec.csv'):
    # load calspec
    stars = pd.read_csv(fName)

    r = []
    stars_all = stars.dropna(subset=['RA', 'Decl'])
    print(len(stars_all),len(stars))
    for i,row in stars_all.iterrows():
        ra = row['RA']
        dec = row['Decl']
        ra = ra.split(' ')
        dec = dec.split(' ')
        name = row['Star_name']
        if len(ra) == 3:
            rr = get_ra_dec(ra,dec)
        else:
            rr = get_ra_dec_from_name(row)
        
        if rr is not None:
            r.append((name,rr.ra.deg,rr.dec.deg))
    stars_df = pd.DataFrame(r, columns=['Star_name','RA','Dec'])
    return stars_df

In [ ]:
# select DDF
def select_DDF(data):
    idx = data.note.str.contains('DD:')
    return pd.DataFrame(data[idx])

In [ ]:
def angular_distance(stars_df,ddf):

    from astropy import (units as u, constants as const)
    c2 = SkyCoord(stars_df['RA'].to_list()*u.degree,stars_df['Dec'].to_list()*u.degree,frame='icrs')
    names = ddf['note'].unique()
    #for i, row in ddf.iterrows():
    r = []
    for name in names:
        iid = ddf['note'] == name
        row = ddf[iid]
        c1 = SkyCoord(row['RA']*u.degree,row['Dec']*u.degree,frame='icrs')
        diff = c1.separation(c2)*u.degree
        print(name,row['RA'].values[0],row['Dec'].values[0],np.min(diff))
        r.append((name,row['RA'].values[0],row['Dec'].values[0],np.min(diff)))
    res = pd.DataFrame(r, columns=['note','RA','Dec','min_sep'])
    return res

In [ ]:
def get_healpixID(nside, stars):

    # get healpixID
    RA = stars['RA'].to_list()
    Dec = stars['Dec'].to_list()
    healpixID = hp.ang2pix(nside, RA,Dec, nest=True, lonlat=True)

    return healpixID
    
    

In [ ]:
def intersect(nside,RA,Dec,width,nradius,healpixIDs_cal):
    # get healpixID
    healpixID = hp.ang2pix(nside, RA,Dec, nest=True, lonlat=True)
    # get 3D vec
    vec = hp.pix2vec(nside, healpixID, nest=True)
    # get nearby pixels
    healpixIDs = hp.query_disc(nside, vec, nradius*np.deg2rad(width), inclusive=True, nest=True)
    in_view = list(set(healpixIDs) & set(healpixIDs_cal))

    return in_view

In [ ]:
def get_radius(RA = 150.09,Dec = 2.16, width = np.sqrt(9.6/np.pi),nside = 128,healpixIDs_cal=pd.DataFrame()):

    for nradius in np.arange(0.1,30.,0.1):
        inter = intersect(nside,RA,Dec,width,nradius,healpixIDs_cal)
        if len(inter) > 0:
            return nradius
    return None

In [ ]:
def plot_pixels(data, fig=None, rot=(0., 0., 0.)):
    """
    Function to plot pixels with weight >= 0

    Parameters
    ----------
    data : pandas df
        data to plot.

    Returns
    -------
    None.

    """
    from matplotlib.colors import LinearSegmentedColormap
    """
    npixels = len(data)
    hpxmap = np.zeros(npixels, dtype=np.float)
    hpxmap = np.full(hpxmap.shape, 0.)
    hpxmap[data['healpixID']] += data['weight']
    xmin = 0.0
    xmax = np.max(data['weight'])
    norm = plt.cm.colors.Normalize(xmin, xmax)

    cmap = plt.cm.jet
    """
    xmin = 0.0
    xmax = np.max(hpxmap)
    norm = plt.cm.colors.Normalize(xmin, xmax)
    colors = ['skyblue', 'yellow','red'] # first color is black, last is blue
    cmap = LinearSegmentedColormap.from_list("Custom", colors, N=3)
    cmap.set_under('w')

    plt.figure(1)
    m = hp.ma(hpxmap,badval=-1)
    map_to_plot = hp.mollview(m, cmap=cmap, nest=True, flip='astro',
                min=xmin, max=xmax, norm=norm, coord=['C'],
                title='', rot=rot,cbar=False,fig=fig,return_projected_map=True)
    plt.close()
    print('rrr',map_to_plot)
     # build ra, dec meshgrid to plot 2d array
    ra_edge = np.linspace(-180, 180, map_to_plot.shape[1] + 1)
    dec_edge = np.linspace(-90, 90, map_to_plot.shape[0] + 1)

    ra_edge[ra_edge > 180] -= 360    # scale conversion to [-180, 180]
    ra_edge = -ra_edge               # reverse the scale: East to the left

    ra_grid, dec_grid = np.meshgrid(ra_edge, dec_edge)

    plt.figure(figsize=figsize)
    ax = plt.subplot(111, projection=projection)
    plt.subplots_adjust(left=0.14, bottom=0.18, right=0.96, top=0.90)
    #colors = ['lightskyblue', 'black',(1, 0, 0)] # first color is black, last is blue
    colors = ['lightskyblue', 'yellow','red']
    cmap = LinearSegmentedColormap.from_list("Custom", colors, N=3)
    mesh = plt.pcolormesh(np.radians(ra_grid), np.radians(dec_grid), map_to_plot, cmap=cmap, edgecolor='none', lw=0)
    hp.graticule(coord='C')

In [ ]:
def get_map(nside):
    """
    Function to get a pixel map

    Parameters
    ----------
    nside : int
        nside healpix parameter.

    Returns
    -------
    map_pixel : pandas df
        cols: pix_RA, pix_Dec, weight, healpixID.

    """

    # get the total number of pixels
    npixels = hp.nside2npix(nside)

    # pixels = hp.get_all_neighbours(nside, 0.0, 0.0, nest=True, lonlat=Tru)

    # get the (RA, Dec) of the pixel centers
    vec = hp.pix2ang(nside, range(npixels), nest=True, lonlat=True)

    map_pixel = pd.DataFrame(range(npixels), columns=['healpixID'])
    map_pixel['pixRA'] = vec[0]
    map_pixel['pixDec'] = vec[1]
    map_pixel['weight'] = 1

    return map_pixel

In [ ]:
dbDir = '../../DB_Files'

In [ ]:
dbName = 'baseline_v3.0_10yrs.npy'

In [ ]:
fullpath = '{}/{}'.format(dbDir,dbName)
data = pd.DataFrame(np.load(fullpath))
nside=32

In [ ]:
# load calspec
stars = get_calspec()

In [ ]:
stars[['Star_name','RA','Dec']]

In [ ]:
ddf = select_DDF(data)

In [ ]:
print(len(ddf))
ddfb = ddf.groupby(['note'])['RA','Dec'].mean().reset_index()

In [ ]:
resa = angular_distance(stars,ddfb)

In [ ]:
healpixID_cal = get_healpixID(nside,stars)
for i, row in resa.iterrows():
    rad = get_radius(RA=row['RA'],Dec=row['Dec'],healpixIDs_cal=healpixID_cal)
    print(row['note'],rad)

In [ ]:
def plot_moll(healpix_map,
              label,
              rot=120,
              cmap='jet',
              projection='mollweide', 
              figsize=(12.0, 9.0), 
              xpad=1.25, 
              labelpad=-37, 
              xlabel_labelpad=10.0, 
              ycb_pos=-0.15, 
              show_legend = True,
              ttext=[]
              ):
    import matplotlib.cm as cm
    from matplotlib.colors import LinearSegmentedColormap
    # transform healpix map to 2d array
    plt.figure(1)
    m = hp.ma(healpix_map,badval=-1)
    map_to_plot = hp.cartview(m, nest=True, rot=rot, flip='geo', fig=1, return_projected_map=True,badcolor='white')
    #map_to_plot = hp.mollview(m, nest=True, rot=rot,flip='geo',fig=1,return_projected_map=True)
    hp.graticule()
    plt.close()

    print(map_to_plot)
    # build ra, dec meshgrid to plot 2d array
    ra_edge = np.linspace(-180, 180, map_to_plot.shape[1] + 1)
    dec_edge = np.linspace(-90, 90, map_to_plot.shape[0] + 1)

    ra_edge[ra_edge > 180] -= 360    # scale conversion to [-180, 180]
    ra_edge = -ra_edge               # reverse the scale: East to the left

    ra_grid, dec_grid = np.meshgrid(ra_edge, dec_edge)

    plt.figure(figsize=figsize)
    ax = plt.subplot(111, projection=projection)
    plt.subplots_adjust(left=0.14, bottom=0.18, right=0.96, top=0.90)
    #colors = ['lightskyblue', 'black',(1, 0, 0)] # first color is black, last is blue
    colors = ['lightskyblue', 'yellow','red']
    cmap = LinearSegmentedColormap.from_list("Custom", colors, N=3)
    mesh = plt.pcolormesh(np.radians(ra_grid), np.radians(dec_grid), map_to_plot, cmap=cmap, edgecolor='none', lw=0)

    tick_labels = np.array([150, 120, 90, 60, 30, 0, 330, 300, 270, 240, 210])
    tick_labels = np.remainder(tick_labels + 360 + rot, 360)
    tick_labels = np.array([f'{lab}°' for lab in tick_labels])
    ax.set_xticklabels(tick_labels)

    ax.set_xlabel('R.A. [deg]', labelpad=xlabel_labelpad)
    ax.xaxis.set_label_position('top')
    ax.set_ylabel('Dec. [deg]')
    for tt in ttext:
        ax.text(tt[0],tt[1],tt[2], fontsize=10, color='red')
    #plt.colorbar()
    plt.savefig('DDF_calspec.png')
    color_map = cm.get_cmap(cmap)
    values_map = np.unique(healpix_map)[~np.isnan(np.unique(healpix_map))]  
    if show_legend:
        legend_elements = [
            Line2D([],[],
                color= color_map((values_map[i] - np.min(values_map)) / (np.max(values_map) - np.min(values_map))),
                label= label[i],
                linestyle="-",
            )
            for i in range(len(values_map))
        ]  
        ax.legend(handles=legend_elements)

In [ ]:
#%matplotlib inline
print('nside',nside)
mm = get_map(nside=nside)
idx = mm['healpixID'].isin(healpixID_cal)
mm.loc[idx,'weight'] = 1
mm.loc[~idx,'weight'] = -1
healpixID_ddf = np.unique(get_healpixID(nside, ddf))
idxb = mm['healpixID'].isin(healpixID_ddf)
mm.loc[idxb,'weight'] = 2
#mm.loc[~idxb,'weight'] = -1
#plot_pixels(mm)

npixels = hp.nside2npix(nside)
hpxmap = np.zeros(npixels,dtype=np.int)
hpxmap = np.full(shape=npixels,fill_value=7.5,dtype=np.int)
print(hpxmap,len(hpxmap))
print(healpixID_cal)
hpxmap[healpixID_cal] = 13
hpxmap[healpixID_ddf] = 20
#plot_pixels(hpxmap)
ttext = [(0.5,10.8,'COSMOS')]
ttext.append((-1.2,10.5,'XMM-LSS'))
ttext.append((-2.0,9.3,'ELAIS'))
ttext.append((-1.3,9.8,'CDFS'))
ttext.append((-0.6,9.,'EDFS'))
plot_moll(hpxmap,'test',show_legend=False,ttext=ttext)